# 协同过滤 ( Collaborative filtering )
> 协同过滤是间接推荐的算法：举例基于用户的协同过滤推荐算法，我们先计算相似用户，然后利用相似度高的用户协同推荐出最受欢迎的 n 个商品，这就是协同过滤。

协同过滤常用于推荐系统。MLlib 提供的 ***ALS*** 目的是填补 user-item 矩阵中缺失项。MLlib 目前支持基于协同过滤模型的交替最小二乘法（ALS）算法来预测缺失的item。具体做法是将矩阵 A 分解成 P 和 Q 矩阵，这样 P * Q = R, R≈A，也就是 R 矩阵是我们填补了 A 矩阵缺失值。所以我们给用户推荐 item 的时候可以推荐那些 R 矩阵中评分比较高且不在 A 矩阵评分项中的 item。具体分解算法是通过最小二乘法实现的可以参考： [ALS](../../数学基础/ALS.md)  
在MLlib 中的实现 ALS 具体要设置以下参数  
* ***numBlocks*** 是用于并行计算的块的数量(设置 －1 是自动模式).
* ***rank*** 模型隐因子数量
* ***iterations*** 算法迭代次数
* ***lambda*** 指定在ALS正则化参数
* ***implicitPrefs*** 指定使用显式反馈数据还是使用隐式反馈数据。
* ***alpha*** is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

## 显式与隐式反馈(Explicit vs. implicit feedback)
基于协同过滤掉标准方法是将 user-item 中的评分项作为用户的显式偏好。  
实际情况下隐式反馈数据更加常见（如 意见，点击，购买，点赞，分享 等）。MLlib 通过协同过滤来处理这样的隐式反馈数据
，本质上，而不是试图直接模拟矩阵的评级。这种方法通过设置不同数据的权重让将他们组合，通过该模型找出潜在的因素，就可以利用该模型来预测一个 item 的用户预期偏好了。
## （正则化参数）Scaling of the regularization parameter
Since v1.1, we scale the regularization parameter lambda in solving each least squares problem by the number of ratings the user generated in updating user factors, or the number of ratings the product received in updating product factors. This approach is named “ALS-WR” and discussed in the paper “Large-Scale Parallel Collaborative Filtering for the Netflix Prize”. It makes lambda less dependent on the scale of the dataset. So we can apply the best parameter learned from a sampled subset to the full dataset and expect similar performance.

## Examples
In the following example we load rating data. Each row consists of a user, a product and a rating. We use the default ALS.train() method which assumes ratings are explicit. We evaluate the recommendation model by measuring the Mean Squared Error of rating prediction.


In [2]:
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
import org.apache.spark.mllib.recommendation.Rating

// Load and parse the data
val data = sc.textFile("../data/mllib/als/test.data")
val ratings = data.map(_.split(',') match { case Array(user, item, rate) =>
  Rating(user.toInt, item.toInt, rate.toDouble)
})

// Build the recommendation model using ALS
val rank = 10
val numIterations = 10
val model = ALS.train(ratings, rank, numIterations, 0.01)

// Evaluate the model on rating data
val usersProducts = ratings.map { case Rating(user, product, rate) =>
  (user, product)
}
val predictions =
  model.predict(usersProducts).map { case Rating(user, product, rate) =>
    ((user, product), rate)
  }
val ratesAndPreds = ratings.map { case Rating(user, product, rate) =>
  ((user, product), rate)
}.join(predictions)
val MSE = ratesAndPreds.map { case ((user, product), (r1, r2)) =>
  val err = (r1 - r2)
  err * err
}.mean()
println("Mean Squared Error = " + MSE)

// Save and load model
model.save(sc, "target/tmp/myCollaborativeFilter")
val sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")

Mean Squared Error = 7.653179116888293E-6


If the rating matrix is derived from another source of information (e.g., it is inferred from other signals), you can use the trainImplicit method to get better results.

In [ ]:
val alpha = 0.01
val lambda = 0.01
val model = ALS.trainImplicit(ratings, rank, numIterations, lambda, alpha )

In order to run the above application, follow the instructions provided in the Self-Contained Applications section of the Spark Quick Start guide. Be sure to also include spark-mllib to your build file as a dependency.